In [10]:
import pandas as pd
from tqdm import tqdm
from geopy.geocoders import Nominatim
import requests


### PUSSY

In [11]:
json = requests.get('https://api.hh.ru/metro/').json()[0]

d = []
for line in json['lines']:
    for station in line['stations']:
        l = [station['name'],station['line']['name'], (station['lat'],station['lng']) ]
        d.append(l)
        d = d.copy()

metro = pd.DataFrame(d).rename(columns={0:'metro',1:'line',2:'coords'})

In [12]:
# metro.head(5)

# metro.to_pickle('metro.pkl')

metro = pd.read_pickle('metro.pkl')

In [13]:
out = pd.read_csv('output/3_4_5_6_rooms.csv').drop_duplicates(subset=['id'])
# out = pd.read_csv('day2_output.csv').drop_duplicates(subset=['id'])

In [14]:
out['coord_house'] = tuple(zip(out['latitude'], out['longitude']))
# test = out.iloc[:3, :].copy()

import geopy.distance as dis

def dist(c1,c2):
    return dis.geodesic(c1, c2).km

def close(coords):
    
        metro_ = metro.copy()
        metro_['dist'] = [dist(coords, metro_i) for metro_i in metro_['coords']]
        
        return metro_.loc[metro_['dist']==min(metro_['dist'])].metro.to_list()[0]

out['metro_station'] = [close(coords) if name == 'Нет данных' else name for coords, name in tqdm(out[['coord_house','metro']].itertuples(index=False))]

3718it [00:04, 818.80it/s] 


In [15]:
out['metro_station'].value_counts()

metro_station
Бунинская аллея           99
Филатов Луг               99
Коммунарка                82
Рассказовка               78
Ольховая                  65
                          ..
Лермонтовский проспект     1
Верхние Котлы              1
Андроновка                 1
Серп и Молот               1
Театральная                1
Name: count, Length: 314, dtype: int64

In [18]:
import string
def strr(x):
    x = x.lower()
    x = x.replace('ё','е')
    for i in string.punctuation + 'iI':
        x = x.replace(i, '')
    
    x = x.replace('зеленоград — ','')
    x = x.replace('аэропорт внуково','внуково')
    x = x.replace('бульвар генерала карбышева','народное ополчение')
    x = x.replace('троицк','ольховая')
    x = x.replace('ватутинки','ольховая')
    x = x.replace('десна','ольховая')
    x = x.replace('кедровая','ольховая')
    x = x.replace('москватоварная','москва товарная')
    x = x.replace('остров мечты','народное ополчение')
    x = x.replace('карамышевская','народное ополчение')
    x = x.replace('матвеевская','аминьевская')
    x = x.replace('летово','ольховая')
    x = x.replace('москвасити','деловой центр')
    x = x.replace('звенигородская','кунцевская')
    x = x.replace('бачуринская','ольховая')
    x = x.replace('университет дружбы народов','югозападная')
    x = x.replace('потапово','бунинская аллея')
    x = x.replace('каспийская','царицыно')
    x = x.replace('гольяново','щелковская')
    x = x.replace('кавказский бульвар','царицыно')
    x = x.replace('тютчевская','коньково')
    # x = x.replace('каспийская','царицыно')

    x = x.strip()

    

    return x

out['metro_m']=out['metro_station'].map(strr)
metro['metro_m']=metro['metro'].map(strr)


dfa = out.drop_duplicates(subset=['id'])
dfb = metro.drop_duplicates(subset=['metro'])

new_df = pd.merge(dfa, dfb, how='left', on='metro_m')

In [19]:
new_df.loc[new_df.metro_y.isna()].metro_m.value_counts()

Series([], Name: count, dtype: int64)

In [42]:
import geopy.distance as dis
def dist_maker_df(df: pd.DataFrame):

    dists = {'metro_dist': (0, 0),
 'domodedovo_dist': (55.425398, 37.892624),
 'sherem_dist': (55.979653, 37.414202),
 'kremlin_dist': (55.752004, 37.617734),
 'izmaylovo_dist': (55.792827, 37.76225)}
    
    def dist_maker(c1, c2):
        return dis.geodesic(c1, c2).km
        
    
    for place, coord in tqdm(dists.items()):
        if place == 'metro_dist':
            
            df[place] = [dist_maker(c1, c2) for c1, c2 in tqdm(df[['coord_house','coords']].itertuples(index=False))]
        else:
            
            df[place] = [dist_maker(c1, coord) for c1 in tqdm(df['coord_house'])]

In [43]:
dist_maker_df(new_df)

  0%|                                                                                                        | 0/5 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1413it [00:00, 14122.94it/s]
2863it [00:00, 14339.07it/s]
4331it [00:00, 14490.37it/s]
5813it [00:00, 14619.43it/s]
7824it [00:00, 14520.96it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.86it/s]


In [44]:
new_df.columns

Index(['offer_type', 'city', 'okrug', 'raion', 'street', 'house', 'latitude',
       'longitude', 'id', 'phone', 'flat_type', 'is_apartment', 'is_penthause',
       'total_area', 'living_area', 'kitchen_area', 'all_rooms_area',
       'combined_wcs_count', 'repair_type', 'floor', 'price', 'currency',
       'material_type', 'floors_count', 'build_year', 'ceiling_height',
       'house_material_type', 'edit_time', 'publication_date',
       'house_material_bti', 'house_heating_supply', 'is_emergency',
       'house_overlap_type', 'metro_x', 'metro_time', 'travel_type',
       'coord_house', 'metro_station', 'metro_m', 'metro_y', 'line', 'coords',
       'metro_dist', 'domodedovo_dist', 'sherem_dist', 'kremlin_dist',
       'izmaylovo_dist'],
      dtype='object')

In [45]:
drop = ['offer_type', 'city','street', 'house',
       'room_type', 'loggiasCount','phone',
       'living_area', 'kitchen_area', 'all_rooms_area',
    'house_material_type',
       'edit_time','house_material_bti',
     'is_emergency', 'house_overlap_type', 'metro_x','metro_m','metro_y',
       'metro_time', 'travel_type', 'coord_house', 'metro_station']

drop2 = ['offer_type', 'city','street', 'house',
       'phone',
       'living_area', 'kitchen_area', 'all_rooms_area',
    'house_material_type',
       'edit_time','house_material_bti',
     'is_emergency', 'house_overlap_type', 'metro_x','metro_m','metro_y',
       'metro_time', 'travel_type', 'coord_house', 'metro_station']

try:
    new_df.set_index('id').drop(drop, axis=1).to_pickle('data2.pkl')
except:
    new_df.set_index('id').drop(drop2, axis=1).to_pickle('data.pkl')

In [46]:
df1 = pd.read_pickle('data.pkl')

In [48]:
df1.head()

,okrug,raion,latitude,longitude,flat_type,is_apartment,is_penthause,total_area,combined_wcs_count,repair_type,...,ceiling_height,publication_date,house_heating_supply,line,coords,metro_dist,domodedovo_dist,sherem_dist,kremlin_dist,izmaylovo_dist
id,,,,,,,,,,,,,,,,,,,,,
287674250,ТАО (Троицкий),пос. Первомайское,55.536797,37.156377,studio,True,False,25.0,1,euro,...,3.0,1684755454,autonomousBoiler,Солнцевская,"(55.6324, 37.3328)",15.396892,48.173650,51.895325,37.655379,47.605832
300158636,ЗАО,район Можайский,55.725615,37.441008,rooms,False,False,36.0,0,no,...,Нет данных,1711647085,central,Арбатско-Покровская,"(55.730673, 37.446522)",0.661175,43.915882,28.334155,11.482907,21.511159
300046208,НАО (Новомосковский),Филимонковское поселение,55.551955,37.344485,studio,True,False,29.0,1,euro,...,Нет данных,1711439159,Нет данных,Сокольническая,"(55.5997, 37.4075)",6.637304,37.405118,47.819502,28.141796,37.551438
299724453,ВАО,район Измайлово,55.791592,37.791109,studio,True,False,11.0,0,euro,...,Нет данных,1710760158,itp,Арбатско-Покровская,"(55.787713, 37.779896)",0.825409,41.268996,31.538971,11.739818,1.815457
300190073,САО,район Коптево,55.835503,37.529196,rooms,False,False,47.3,0,cosmetic,...,Нет данных,1711713265,central,МЦК,"(55.839637, 37.520037)",0.735633,51.075820,17.587371,10.829062,15.363768


In [51]:
metro.loc[metro.metro.str.contains('юберцы')]

,metro,line,coords,metro_m
393,Люберцы I,МЦД-3,"(55.681667, 37.896944)",люберцы


In [50]:
out.loc[out.metro.str.contains('арбыш')]

,offer_type,city,okrug,raion,street,house,latitude,longitude,id,phone,...,house_material_bti,house_heating_supply,is_emergency,house_overlap_type,metro,metro_time,travel_type,coord_house,metro_station,metro_m


### HARDCORE

In [7]:
import geopy.distance as dis
import pandas as pd
import string
import numpy as np
import datetime
from tqdm import tqdm
import requests

from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin


import sklearn
sklearn.set_config(transform_output="pandas")
import warnings
warnings.filterwarnings('ignore')

def strr(x):
    x = x.lower()
    x = x.replace('ё','е')
    
    for i in string.punctuation + 'iI':
        x = x.replace(i, '')
    
    x = x.replace('зеленоград — ','')
    x = x.replace('аэропорт внуково','внуково')
    x = x.replace('бульвар генерала карбышева','народное ополчение')
    x = x.replace('троицк','ольховая')
    x = x.replace('ватутинки','ольховая')
    x = x.replace('десна','ольховая')
    x = x.replace('кедровая','ольховая')
    x = x.replace('москватоварная','москва товарная')
    x = x.replace('остров мечты','народное ополчение')
    x = x.replace('карамышевская','народное ополчение')
    x = x.replace('матвеевская','аминьевская')
    x = x.replace('летово','ольховая')
    x = x.replace('москвасити','деловой центр')
    x = x.replace('звенигородская','кунцевская')
    x = x.replace('бачуринская','ольховая')
    x = x.replace('университет дружбы народов','югозападная')
    x = x.replace('потапово','бунинская аллея')
    x = x.replace('каспийская','царицыно')
    x = x.replace('гольяново','щелковская')
    x = x.replace('кавказский бульвар','царицыно')
    x = x.replace('тютчевская','коньково')

    x = x.strip()
    return x


try:
    url = 'https://www.cbr-xml-daily.ru/latest.js'
    res = requests.get(url)
    usd = 1/res.json()['rates']['USD']
    eur = 1/res.json()['rates']['EUR']
except:
    usd = 92
    eur = 100

def dist(c1,c2):
    return dis.geodesic(c1, c2).km

def dist_maker_df(df: pd.DataFrame):
    dists = {'metro_dist': (0.0, 0.0),
 'domodedovo_dist': (55.425398, 37.892624),
 'sherem_dist': (55.979653, 37.414202),
 'kremlin_dist': (55.752004, 37.617734),
             'moskva_city_dist': (55.749776, 37.536234),
             'mgu_dist': (55.703589, 37.530797),
             'nekrasovka_dist': (55.704123, 37.926320),
             'strogino_dist': (55.800774, 37.416551),
 'izmaylovo_dist': (55.792827, 37.76225)}
    
    for place, coord in tqdm(dists.items()):
        
        if place == 'metro_dist':
            
            df[place] = [dist(c1, c2) for c1, c2 in tqdm(df[['coord_house','coords']].itertuples(index=False))]
        else:
            
            df[place] = [dist(c1, coord) for c1 in tqdm(df['coord_house'])]


class Coord_imp(BaseEstimator, TransformerMixin): 
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X: pd.DataFrame, y=None):
        X_copy = X.copy()
        X_copy['coord_house'] = tuple(zip(X_copy['latitude'].astype(float), X_copy['longitude'].astype(float)))

        return X_copy

class Metro_Names(BaseEstimator, TransformerMixin): 
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X: pd.DataFrame, y=None):

        def close(coords):
    
            metro_ = metro.copy()
            metro_['dist'] = [dist(coords, metro_i) for metro_i in metro_['coords']]
            return metro_.loc[metro_['dist']==min(metro_['dist'])].metro.to_list()[0]
        
        X_copy = X.copy()
        X_copy['metro_station'] = [close(coords) if name == 'Нет данных' else name for coords, name in tqdm(X_copy[['coord_house','metro']].itertuples(index=False))]

        return X_copy

class Metro_Normalizer(BaseEstimator, TransformerMixin): 
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X: pd.DataFrame, y=None):
        
        X_copy = X.copy()
        metro_copy = metro.copy()
        

        X_copy['metro_m']=X_copy['metro_station'].map(strr)
        metro_copy['metro_m']=metro_copy['metro'].map(strr)

        metro_copy = metro_copy.drop_duplicates(subset=['metro'])

        X_copy_2 = pd.merge(X_copy, metro_copy, how='left', on='metro_m')

        return X_copy_2


class Dist_Maker(BaseEstimator, TransformerMixin): 
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X: pd.DataFrame, y=None):
        X_copy = X.copy()

        dist_maker_df(X_copy)

        return X_copy


class SmartDropper(BaseEstimator, TransformerMixin): 
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X: pd.DataFrame, y=None):
        X_copy = X.copy()
        
        drop_1 = ['offer_type', 'city','street', 'house','phone','raion',
               'room_type', 'loggiasCount',
               'living_area', 'kitchen_area', 'all_rooms_area', 'house_material_type',
               'edit_time','house_material_bti', 'is_emergency', 'house_overlap_type', 'metro_x','metro_y',
               'metro_time', 'travel_type', 'coord_house', 'metro_station']

        drop_2 = ['offer_type', 'city','street', 'house', 'phone','raion',
               'living_area', 'kitchen_area', 'all_rooms_area','house_material_type',
               'edit_time','house_material_bti', 'is_emergency', 'house_overlap_type', 'metro_x','metro_y',
               'metro_time', 'travel_type', 'coord_house', 'metro_station']

        

        try:
            X_copy = X_copy.drop(drop_1, axis=1)
        except:
            X_copy = X_copy.drop(drop_2, axis=1)


        return X_copy

class Some_transformations(BaseEstimator, TransformerMixin): 
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X: pd.DataFrame, y=None):
        X_copy = X.copy()
        def heat_func(x):
            if x == 'Нет данных':
                return 'central'
            return x
        
        X_copy['house_heating_supply'] = X_copy['house_heating_supply'].map(heat_func)

        def apartment(x):
            if x in ['Нет данных','False',False]:
                return 0
            return 1
            
        X_copy['is_apartment'] = X_copy['is_apartment'].map(apartment)

        cur = {'rub':1,
                   'rur':1,
                  'usd': usd,
                  'eur' : eur,
                  'euro' : eur}
        
        X_copy['currency'] = X_copy['currency'].map(cur)
        X_copy['price'] = X_copy['price'].astype(float) *  X_copy['currency'].astype(float)
            

        return X_copy

In [15]:
class LargeImputers(BaseEstimator, TransformerMixin): 
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X: pd.DataFrame, y=None):
        X_copy = X.copy()

        def rooms(area):
            if area <= 40:
                return 1
            elif area <= 80:
                return 2
            elif area <= 120:
                return 3
            elif area <= 160:
                return 4
            return 5
        
        X_copy['total_area'] = X_copy['total_area'].astype(float)
        
        X_copy['room_count'] = [rooms(area) if np.isnan(room) or room in [0,'0','False',False,'Нет данных'] else room for area, room in X_copy[['total_area','room_count']].itertuples(index=False)]

        height = X_copy.loc[X_copy.ceiling_height != 'Нет данных'].ceiling_height.astype(float).median()

        def ceil(x):
            if x == 'Нет данных':
                return height
            x = float(x)

            if x <= 1.5:
                return height
            return x

        X_copy['ceiling_height'] = X_copy['ceiling_height'].map(ceil)

        def f(year):
            if year == 'Нет данных':
                return np.NaN
            else:
                return int(year)
                
        X_copy['build_year'] = X_copy['build_year'].map(f)

        def ff(x):
            if x == 'боровицкая':
                x = 'арбатская'
            return metro_year.loc[metro_year.index == x].iloc[0].item()
        
        
        X_copy['build_year'] = [ff(station) if year <= 1500 or np.isnan(year) or year in ['Нет данных ',0,'0',False] else year for year, station in X_copy[['build_year','metro_m']].itertuples(index=False)]

        def repair_imp(x):
            if x in ['Нет данных',0,'0',False] or pd.isna(x):
                return 'no'
            return x
        
        X_copy['repair_type'] = X_copy['repair_type'].map(repair_imp)
        return X_copy



class Material_IMP(BaseEstimator, TransformerMixin): 
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X: pd.DataFrame, y=None):
        X_copy = X.copy()

        def typer(year):
            if year <= 1963:
                return 'brick'
            elif year <= 1995:
                return 'panel'
            return 'monolith'
        
        X_copy['material_type'] = [typer(year) if mtype in [0,'0','False',False,'Нет данных'] or pd.isna(mtype) else mtype for year, mtype in X_copy[['build_year','material_type']].itertuples(index=False)]

        return X_copy


class Okrug_Normalizer(BaseEstimator, TransformerMixin): 
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X: pd.DataFrame, y=None):
        X_copy = X.copy()

        def func(station):
            okr = metro_okrug.loc[metro_okrug.metro_m == station].new_okrug.to_list()[0]
            if type(okr)==str:
                return okr
            return okr[0]

        X_copy.okrug = [func(station) if "АО" not in distr else distr for station, distr in X_copy[['metro_m','okrug']].itertuples(index=False)]
        return X_copy

In [16]:
coord_list = ['latitude','longitude']
metro_list = ['coord_house','metro']
final_stage = ['house_heating_supply','is_apartment','currency','price']
to_drop = ['currency','coords']

first_step = ColumnTransformer(
    transformers=[
        ('house_coord', Coord_imp(), coord_list),
    ],
     verbose_feature_names_out = False,
     remainder = 'passthrough')

second_step = ColumnTransformer(
    transformers=[
        ('metro_names', Metro_Names(), metro_list),
    ],
     verbose_feature_names_out = False,
     remainder = 'passthrough')

third_step = ColumnTransformer(
    transformers=[
        ('metro_normalize', Metro_Normalizer(), make_column_selector()),
    ],
     verbose_feature_names_out = False,
     remainder = 'passthrough')


fourth_step = ColumnTransformer(
    transformers=[
        ('dist_maker', Dist_Maker(), make_column_selector()),
    ],
     verbose_feature_names_out = False,
     remainder = 'passthrough')


fifth_step = ColumnTransformer(
    transformers=[
        ('dropper', SmartDropper(), make_column_selector()),
    ],
     verbose_feature_names_out = False,
     remainder = 'passthrough')

sixth_step = ColumnTransformer(
    transformers=[
        ('some_ltl_transformations', Some_transformations(), final_stage),
    ],
     verbose_feature_names_out = False,
     remainder = 'passthrough')

seventh_step = ColumnTransformer(
    transformers=[
        ('final_dropper', 'drop', to_drop),
    ],
     verbose_feature_names_out = False,
     remainder = 'passthrough')

preprocessor = Pipeline(
    [
        ('coord_maker', first_step),
        ('metro_names_maker', second_step),
        ('metro_normalizer', third_step),
        ('distance_maker', fourth_step),
        ('smart_dropper', fifth_step),
        ('little_transforms', sixth_step),
        ('dropper_2', seventh_step)
    ]
)

In [17]:
last_stage = ['total_area','room_count','ceiling_height','build_year','metro_m','repair_type']

first_step = ColumnTransformer(
    transformers=[
        ('diffrent_imputers', LargeImputers(), last_stage),
    ],
     verbose_feature_names_out = False,
     remainder = 'passthrough')

second_stage = ColumnTransformer(
    transformers=[
        ('drop', 'drop', ['publication_date']),
        ('diffrent_imputers', Material_IMP(), ['build_year','material_type']),
        ('okrug_norm', Okrug_Normalizer(), ['okrug','metro_m']),
    ],
     verbose_feature_names_out = False,
     remainder = 'passthrough')


preprocessor_stage_2 = Pipeline(
    [
        ('some large transformations', first_step),
        ('drop and material', second_stage)
    ]
)

metro = pd.read_pickle('metro.pkl')
metro_year = pd.read_pickle('metro_year.pkl')
metro_okrug = pd.read_pickle('metro_okrug.pkl')

### MAIN

In [5]:
#1

data = pd.read_csv('output.csv').drop_duplicates(subset=['id']).set_index('id')
data2 = pd.read_csv('day2_output.csv').drop_duplicates(subset=['id']).set_index('id')

#3 preprocessor

data_pre = preprocessor.fit_transform(data)
data_pre.index = data.index.to_list()


data_pre2 = preprocessor.fit_transform(data2)
data_pre2.index = data2.index.to_list()



#4 making 27 columns

if 'room_count' not in data_pre.columns:
    data_pre['room_count'] = np.NaN


if 'room_count' not in data_pre2.columns:
    data_pre2['room_count'] = np.NaN



#5 preprocessor_2

data_pre_2 = preprocessor_stage_2.fit_transform(data_pre)
data_pre2_2 = preprocessor_stage_2.fit_transform(data_pre2)


#5 saving
# ct = datetime.datetime.now()
# title = f'data_{str(ct)[:str(ct).find(".")].replace(" ","_")}.pkl'

# data_pre.to_pickle(title)

6070it [00:00, 3282545.81it/s]
  0%|                                                                                                        | 0/9 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1583it [00:00, 15825.00it/s]
3235it [00:00, 16227.86it/s]
6070it [00:00, 16452.59it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:03<00:00,  2.63it/s]
15228it [00:16, 926.17it/s] 
  0%|                                                                                                        | 0/9 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1690it [00:00, 16897.12it/s]
3380it [00:00, 16754.35it/s]
5056it [00:00, 16635.79it/s]
6720it [00:00, 16605.39it/s]
8381it [00:00, 16581.04it/s]
10040it [00:00, 16551.18it/s]
11696it [00:00, 16125.25it/s]
13311it [00:00, 15575.31it/s]
15228it [00:00, 16153.06it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:08<00:00,  1.03it/s]


In [6]:
pd.concat([data_pre_2, data_pre2_2]).to_pickle('data_comb.pkl')

In [8]:
# okruga = df.okrug.value_counts().index.to_list()

# 
# df.new_okrug.value_counts().index.to_list()
# metro_okrug = df.groupby('metro_m').agg({'new_okrug':pd.Series.mode}).reset_index()
# metro_okrug.to_pickle('metro_okrug.pkl')

In [6]:
# bins = 20
# df.groupby(pd.cut(df.build_year, bins)).agg({'price':'median','metro_m':'count','material_type':pd.Series.mode})

### NEW_OUTPUT

In [18]:
# 1
data = pd.read_csv('output/3_4_5_6_rooms.csv').drop_duplicates(subset=['id']).set_index('id')

In [19]:

#2 preprocessor

data_pre = preprocessor.fit_transform(data)
data_pre.index = data.index.to_list()

# # #3 making 27 columns

if 'room_count' not in data_pre.columns:
    data_pre['room_count'] = np.NaN

# #4 preprocessor_2
data_pre_2 = preprocessor_stage_2.fit_transform(data_pre)



3718it [00:04, 833.55it/s] 
  0%|                                                                                                        | 0/9 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1690it [00:00, 16894.22it/s]
3718it [00:00, 16492.23it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:02<00:00,  4.23it/s]


In [20]:
data_pre_2.to_pickle('out3.pkl')

In [21]:
df1 = pd.read_pickle('out1.pkl')
df2 = pd.read_pickle('out2.pkl')
df3 = pd.read_pickle('out3.pkl')
df4 = pd.read_pickle('data_comb.pkl')

In [26]:
DF = pd.concat([df1,df2,df3,df4])

In [31]:
DF.to_pickle('data_comb.pkl')